In [1]:
import graphlab
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# carga de datos

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1450267943 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\margrami\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1450267943 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to margrami221@yahoo.com and will expire on October 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3732 - Server binary: C:\Users\margrami\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\margrami\AppData\Local\Temp\graphlab_server_1450267943.log.0
[INFO] GraphLab Server Version: 1.7.1


In [3]:
train_and_validation_data, test_data = sales.random_split(0.9, seed=1)
train_data, val_data = train_and_validation_data.random_split(0.5, seed=1)

In [25]:
def comparar_modelos(set_data, set_val_data, my_features):
    model= graphlab.linear_regression.create(set_data, target = 'price', features = my_features, validation_set = None, verbose = False)
    my_rmse = graphlab.evaluation.rmse(set_val_data['price'], model.predict(set_val_data))
    return my_rmse

# Aqui tengo una gran duda: 
# dado que la hypotesis es es H(w) = w0+ w1x + w2X^2 +...+ wnX^n. Debo evaluar el set val_data en forma polinomial??? 


In [36]:
degree = 15
features_vector = ['sqft_living']
rmse_vector =[comparar_modelos(train_data, val_data, features_vector)]
for i in range(2, degree+1):
    name = 'power_' + str(i)
    features_vector.append(name)
    train_data[name] = train_data['sqft_living'].apply(lambda x: x**i)
    my_rmse = comparar_modelos(train_data, val_data, features_vector)
    rmse_vector.append(my_rmse)
selected = rmse_vector.index(min(rmse_vector)) + 1   
print (rmse_vector)
print ('Polynomial Degree with min rmse is : ' + str(selected))  

Polynomial Degree with min rmse is : 6


# ahora applicando polynomial_Sframe a val_data

In [38]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[str(name)] = feature.apply(lambda x: x**power)
    return poly_sframe

In [43]:
val_data_poly = polynomial_sframe(val_data['sqft_living'], 15)
val_data_poly['price'] = val_data['price']

In [45]:
degree = 15
features_vector = ['sqft_living']
rmse_vector =[comparar_modelos(train_data, val_data_poly, features_vector)]
for i in range(2, degree+1):
    name = 'power_' + str(i)
    features_vector.append(name)
    #train_data[name] = train_data['sqft_living'].apply(lambda x: x**i)
    my_rmse = comparar_modelos(train_data, val_data_poly, features_vector)
    rmse_vector.append(my_rmse)
selected = rmse_vector.index(min(rmse_vector)) + 1   
print (rmse_vector)
print ('Polynomial Degree with min rmse is : ' + str(selected))  

[356204.81970734446, 254396.56780271206, 254436.40259179243, 257103.6131311395, 263623.624317145, 336060.4338609304, 453947.2721422761, 460012.34535565827, 474850.853327861, 497657.2074171463, 495598.1184413642, 448720.03615651524, 370959.0281381711, 387144.66840212955, 673709.7727920217]
Polynomial Degree with min rmse is : 2
